# Check the results of CausalImpact analysis
CausalImpact in R: https://github.com/google/CausalImpact  
tfcausalimpact in Python: https://github.com/WillianFuks/tfcausalimpact

In [ ]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import seaborn as sns

import sys
sys.path.append("../") # Set parent directory to sys.path
sys.dont_write_bytecode = True
%load_ext autoreload
%autoreload 2
import src.utils as utils
import src.ci_utils as ci_utils

palette0 = sns.color_palette(['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7', '#000000']) # Okabe-Ito
palette2 = sns.color_palette(["#D81B60", "#1E88E5", "#FFC107", "#004D40"])
palette = palette0
display(palette)
sns.set_palette(palette)
sns.set_theme(context='poster', style='ticks', palette=palette, font_scale=1.0)

In [ ]:
l = [1, 2, 3, 4, 5]
a = np.array(l)
print(a)
print(a[::-1]) # reverse

## Configuration

In [ ]:
# run_id = "run-05"
run_id = "run-10"
# run_id = "run-15"
# run_id = "run-20"

cfg_path = f"../output/causal-impact/{run_id}.yaml"
ci_cfg = ci_utils.load_from_yaml(cfg_path)
print(f"cfg_path: {cfg_path}")
display(ci_cfg)

## Example Visualization

### Figure 4

In [ ]:
sns.set_theme(context='poster', style='ticks', palette=palette, font_scale=1.0)
# run_id_list = ["run-05"]
run_id_list = ["run-10"]
# run_id_list = ["run-15"]
# run_id_list = ["run-20"]
data_type_list = ["smoothed_VeDBA_2s", "abs_diff_distance_m", "abs_diff_pixel_count_p"]
for run_id in run_id_list:
    # setup
    cfg_path = f"../output/causal-impact/{run_id}.yaml"
    ci_cfg = ci_utils.load_from_yaml(cfg_path)
    print(f"cfg_path: {cfg_path}")

    # plot
    fig = ci_utils.plot_figure_04(data_type_list, ci_cfg)
    
    # Save figure
    _run_id = run_id.replace("-", "_")
    # Change the path appropriately
    save_dir = f"C:/Users/ryoma/D/writing/00-first/005_Otsuka_202x_MEE_umineko_playback/otsuka-umineko-playback/illustration/export"
    fig.savefig(f"{save_dir}/fig_04_{_run_id}_r.png", dpi=600, bbox_inches="tight", pad_inches=1.0, transparent=False)
    fig.savefig(f"{save_dir}/fig_04_{_run_id}_r.pdf", dpi=600, bbox_inches="tight", pad_inches=1.0, transparent=False)
    plt.show()

## Plot for each session

In [ ]:
# just to get the list of _session_id (e.g., LBP03_S08)
run_id = "run-10"
_data_type = "abs_diff_pixel_count_p"
_session_name = "*"
_path_target = f"../output/causal-impact/{run_id}/{_data_type}/{_session_name}"
print(_path_target)
_path_list = sorted(glob.glob(_path_target))
print(len(_path_list))
_session_id_list = [os.path.basename(_path) for _path in _path_list]
print(_session_id_list)
# _session_id_list = _session_id_list[:2] # for debugging
# print(_session_id_list)

In [ ]:
sns.set_theme(context='notebook', style='ticks', palette=palette, font_scale=1.0)
# run_id = "run-05"
run_id = "run-10"
# run_id = "run-15"
# run_id = "run-20"
for s, _session_id in enumerate(_session_id_list):
# for s, _session_id in enumerate(_session_id_list[-2:]):
    
    cfg_path = f"../output/causal-impact/{run_id}.yaml"
    ci_cfg = ci_utils.load_from_yaml(cfg_path)
    print(f"cfg_path: {cfg_path}") if s == 0 else None
    print(f"{s:02d} | {_session_id}")

    data_type_list = ["smoothed_VeDBA_2s", "abs_diff_distance_m", "abs_diff_pixel_count_p"]
    fig = ci_utils.plot_causal_impact_per_session(
        _session_id, data_type_list, ci_cfg
    )
    # Save figure
    save_dir = f"../output/figure/causal-impact/{run_id}-r"
    os.makedirs(save_dir, exist_ok=True)
    fig.savefig(f"{save_dir}/{_session_id}.png", dpi=100, bbox_inches="tight", pad_inches=0.2, transparent=False)
    plt.show()
    plt.close()

## Summary of the Causal Impact analysis

### Figure Sxx

In [ ]:
def setup_df_for_summary(run_id, data_type):
    
    target_path = f"../output/causal-impact/{run_id}/{data_type}/*/r/summary_data_r.csv"
    input_path_list = sorted(glob.glob(target_path))
    # print(len(input_path_list))

    session_name_list = []
    avg_effect_list = []
    avg_effect_lower_list = []
    avg_effect_upper_list = []
    cum_effect_list = []
    cum_effect_lower_list = []
    cum_effect_upper_list = []
    p_value_list = []
    for input_path in input_path_list:
    # for input_path in input_path_list[0:5]:
        session_name = os.path.basename(os.path.dirname(os.path.dirname(input_path)))
        df_summary = pd.read_csv(input_path)
        # display(df_summary)
        avg_effect = df_summary['AbsEffect'].values[0]
        avg_effect_lower = df_summary['AbsEffect.lower'].values[0]
        avg_effect_upper = df_summary['AbsEffect.upper'].values[0]
        cum_effect = df_summary['AbsEffect'].values[1]
        cum_effect_lower =df_summary['AbsEffect.lower'].values[1]
        cum_effect_upper = df_summary['AbsEffect.lower'].values[1]
        p_value = df_summary['p'].values[0]
        # print(f"p-value: {p_value:.3f}")
        session_name_list.append(session_name)
        avg_effect_list.append(avg_effect)
        avg_effect_lower_list.append(avg_effect_lower)
        avg_effect_upper_list.append(avg_effect_upper)
        cum_effect_list.append(cum_effect)
        cum_effect_lower_list.append(cum_effect_lower)
        cum_effect_upper_list.append(cum_effect_upper)
        p_value_list.append(p_value)
    
    data_dict = {
        "file_name": session_name_list,
        "session_name": session_name_list,
        "avg_effect": avg_effect_list,
        "avg_effect_lower": avg_effect_lower_list,
        "avg_effect_upper": avg_effect_upper_list,
        "cum_effect": cum_effect_list,
        "cum_effect_lower": cum_effect_lower_list,
        "cum_effect_upper": cum_effect_upper_list,
        "p_value": p_value_list,
    }
    df = pd.DataFrame(data_dict)

    # load metadata and merge with df
    df_meta = pd.read_csv('../data/metadata/session_data.csv')
    df_meta = df_meta[["file_name", "audio_file_name", "test_id", "pb_count", "pb_count_2"]]
    df = pd.merge(df, df_meta, on='file_name', how='inner')
    # display(df.head(3))

    return df

### 1×3

In [ ]:
# run_id = "run-05"
run_id = "run-10"

sns.set_theme(context='paper', style='ticks', palette=palette, font_scale=1.1) # very good to see
GRIDSPEC_KW = {'wspace': 0.2, 'hspace': 0.2, 'width_ratios': [1, 1, 1]}
fig, axes = plt.subplots(1, 3, figsize=(7, 11), gridspec_kw=GRIDSPEC_KW)
ax_list = axes.flatten().tolist()
data_type_list = ["smoothed_VeDBA_2s",  "abs_diff_distance_m", "abs_diff_pixel_count_p"]
data_name_list = ["S-VeDBA", "AD-Speed", "AD-MPR"]

# R version
for i, data_type in enumerate(data_type_list):
    ax = ax_list[i]
    data_name = data_name_list[i]
    df = setup_df_for_summary(run_id, data_type)

    # plot config
    if data_type == "smoothed_VeDBA_2s":
        xticks = np.arange(-5, 5, 0.5)
        xlim = (-1.2, 1.2)
        COLOR = "#1E88E5"
    elif data_type == "abs_diff_distance_m":
        xticks = np.arange(-5, 5, 1.0)
        xlim = (-2.5, 2.5)
        COLOR = "#FFC107"
    elif data_type == "abs_diff_pixel_count_p":
        xticks = np.arange(-1, 1, 0.02)
        xlim = (-0.035, 0.035)
        COLOR = "#D81B60"
    
    # data for plot | reversed order
    _avg_effects = df['avg_effect'].values[::-1]
    _avg_effect_lowers = df['avg_effect_lower'].values[::-1]
    _avg_effect_uppers = df['avg_effect_upper'].values[::-1]
    _cum_effects = df['cum_effect'].values[::-1]
    _cum_effect_lowers = df['cum_effect_lower'].values[::-1]
    _cum_effect_uppers = df['cum_effect_upper'].values[::-1]
    _session_names = df['session_name'].values[::-1]
    _session_names_with_spaces = [name.replace('_', ' ') for name in _session_names]
    y = np.arange(len(_session_names))

    # print(_session_names)
    # print(y)

    # display(df.head(3))
    
    # colors
    colors = []
    for lower, upper in zip(_avg_effect_lowers, _avg_effect_uppers):
        if lower <= 0 <= upper:
            colors.append('#888888')
        else:
            colors.append(COLOR)
    colors = np.array(colors) # already reversed

    # shapes
    shapes = []
    for j in range(len(df)):
        audio_file_name = df["audio_file_name"][j]
        if audio_file_name == "Predator":
            shapes.append("^")
        else:
            shapes.append("o")
    shapes = np.array(shapes)[::-1] # should be reversed
    
    # Lines and Points
    for j in range(len(y)):
        ax.plot([_avg_effect_lowers[j], _avg_effect_uppers[j]], [y[j], y[j]], color=colors[j], linewidth=1.2, zorder=11)
        ax.scatter(x=_avg_effects[j], y=y[j], color=colors[j], marker=shapes[j], s=30, zorder=12)

    # Points
    # ax.scatter(x=_avg_effects, y=y, color=colors, marker=shapes, s=20, zorder=10)

    # Cumulative effect
    # for j in range(len(y)):
    #     ax.plot([_cum_effect_lowers[j], _cum_effect_uppers[j]], [y[j], y[j]], color=colors[j], linewidth=1.2, zorder=11)
    # ax.scatter(x=_cum_effects, y=y, color=colors, s=20, zorder=10)
    
    # center line (x = 0) 
    ax.axvline(x=0, color="black", linestyle="-", linewidth=1.0, zorder=3)

    # customize plot
    ax.set_xticks(xticks)
    ax.set_xlim(xlim)

    ax.set_yticks(y)
    if i == 0 or i == 3:
        ax.set_yticklabels(_session_names_with_spaces)
    else:
        ax.set_yticklabels([])
    ax.set_ylim(0-1, len(y))

    title = f"{data_name_list[i]}"
    ax.set_title(title, fontweight='bold', pad=7)
    ax.xaxis.set_minor_locator(tck.AutoMinorLocator(2))
    ax.grid(which='major', axis="both", alpha=0.5)
    ax.grid(which='minor', axis="x", alpha=0.5)

plt.show()
_run_id = run_id.replace("-", "_")
save_dir = f"../output/figure-for-paper/"
fig.savefig(f"{save_dir}/pdf/fig_sxx_causal_impact_{_run_id}_r.pdf", dpi=600, bbox_inches="tight", pad_inches=0.2, transparent=False)
fig.savefig(f"{save_dir}/png/fig_sxx_causal_impact_{_run_id}_r.png", dpi=600, bbox_inches="tight", pad_inches=0.2, transparent=False)


### 2×3 Predator & Noise separately

In [ ]:
# run_id = "run-05"
run_id = "run-10"

sns.set_theme(context='paper', style='ticks', palette=palette, font_scale=1.0)
GRIDSPEC_KW = {'wspace': 0.2, 'hspace': 0.2, 'width_ratios': [1, 1, 1], 'height_ratios': [18, 28]}
fig, axes = plt.subplots(2, 3, figsize=(7, 11), gridspec_kw=GRIDSPEC_KW)
ax_list = axes.flatten().tolist()

audio_file_name_list = ["Predator", "Predator", "Predator", "Noise", "Noise", "Noise"]

data_type_list = ["smoothed_VeDBA_2s",  "abs_diff_distance_m", "abs_diff_pixel_count_p"] * 2
data_name_list = ["S-VeDBA", "AD-Speed", "AD-MPR"] * 2

# R version
for i, data_type in enumerate(data_type_list):
    ax = ax_list[i]
    data_name = data_name_list[i]
    df = setup_df_for_summary(run_id, data_type)
    df = df[df["audio_file_name"] == audio_file_name_list[i]] # filter

    # plot config
    if data_type == "smoothed_VeDBA_2s":
        xticks = np.arange(-5, 5, 0.5)
        xlim = (-1.2, 1.2)
        COLOR = "#1E88E5"
    elif data_type == "abs_diff_distance_m":
        xticks = np.arange(-5, 5, 1.0)
        xlim = (-2.5, 2.5)
        COLOR = "#FFC107"
    elif data_type == "abs_diff_pixel_count_p":
        xticks = np.arange(-1, 1, 0.02)
        xlim = (-0.035, 0.035)
        COLOR = "#D81B60"
    
    # data for plot| reversed order
    _avg_effects = df['avg_effect'].values[::-1]
    _avg_effect_lowers = df['avg_effect_lower'].values[::-1]
    _avg_effect_uppers = df['avg_effect_upper'].values[::-1]
    _cum_effects = df['cum_effect'].values[::-1]
    _cum_effect_lowers = df['cum_effect_lower'].values[::-1]
    _cum_effect_uppers = df['cum_effect_upper'].values[::-1]
    _session_names = df['session_name'].values[::-1]
    _session_names_with_spaces = [name.replace('_', ' ') for name in _session_names]
    y = np.arange(len(_session_names))

    # print(_session_names)
    # print(y)
    
    # lines and points
    colors = []
    for lower, upper in zip(_avg_effect_lowers, _avg_effect_uppers):
        if lower <= 0 <= upper:
            colors.append('#888888')
        else:
            colors.append(COLOR)
    colors = np.array(colors)

    # Lines
    for j in range(len(y)):
        ax.plot([_avg_effect_lowers[j], _avg_effect_uppers[j]], [y[j], y[j]], color=colors[j], linewidth=1.2, zorder=11)
    
    # Points
    shape = "^" if audio_file_name_list[i] == "Predator" else "o"
    ax.scatter(x=_avg_effects, y=y, color=colors, marker=shape, s=30, zorder=12)

    # Cumulative effect
    # for j in range(len(y)):
    #     ax.plot([_cum_effect_lowers[j], _cum_effect_uppers[j]], [y[j], y[j]], color=colors[j], linewidth=1.2, zorder=11)
    # ax.scatter(x=_cum_effects, y=y, color=colors, s=20, zorder=10)
    
    # center line (x = 0) 
    ax.axvline(x=0, color="black", linestyle="-", linewidth=1.0, zorder=3)

    # customize plot
    ax.set_xticks(xticks)
    ax.set_xlim(xlim)

    ax.set_yticks(y)
    if i == 0 or i == 3:
        ax.set_yticklabels(_session_names_with_spaces)
    else:
        ax.set_yticklabels([])
    ax.set_ylim(0-1, len(y))

    if i < 3:
        title = f"{data_name_list[i]} | Predator"
    else:
        title = f"{data_name_list[i]} | Noise"
    ax.set_title(title, fontweight='bold', pad=7)
    ax.xaxis.set_minor_locator(tck.AutoMinorLocator(2))
    ax.grid(which='major', axis="both", alpha=0.5)
    ax.grid(which='minor', axis="x", alpha=0.5)

plt.show()
_run_id = run_id.replace("-", "_")
save_dir = f"../output/figure-for-paper/"
fig.savefig(f"{save_dir}/pdf/fig_sxx_causal_impact_{_run_id}_r_sep.pdf", dpi=600, bbox_inches="tight", pad_inches=0.2, transparent=False)
fig.savefig(f"{save_dir}/png/fig_sxx_causal_impact_{_run_id}_r_sep.png", dpi=600, bbox_inches="tight", pad_inches=0.2, transparent=False)

### Table

In [ ]:
# R version
run_id = "run-05"
# run_id = "run-10"

data_type_list = ["smoothed_VeDBA_2s", "abs_diff_distance_m", "abs_diff_pixel_count_p"]
data_name_list = ["S-VeDBA", "AD-Speed", "AD-MPR"]
df = None
for i, data_type in enumerate(data_type_list):
    data_name = data_name_list[i]
    target_path = f"../output/causal-impact/{run_id}/{data_type}/*/r/summary_data_r.csv"
    input_path_list = sorted(glob.glob(target_path))
    print(len(input_path_list))
    session_name_list = []
    avg_effect_list = []
    avg_effect_lower_list = []
    avg_effect_upper_list = []
    p_value_list = []
    mean_lower_upper_list = []
    for input_path in input_path_list:
    # for input_path in input_path_list[0:5]:
        session_name = os.path.basename(os.path.dirname(os.path.dirname(input_path)))
        df_summary = pd.read_csv(input_path)
        # display(df_summary)
        avg_effect = df_summary['AbsEffect'].values[0]
        avg_effect_lower = df_summary['AbsEffect.lower'].values[0]
        avg_effect_upper = df_summary['AbsEffect.upper'].values[0]
        p_value = df_summary['p'].values[0]
        # print(f"p-value: {p_value:.3f}")
        mean_lower_upper = f"{avg_effect:.4f} ({avg_effect_lower:.4f}\u2013{avg_effect_upper:.4f})"
        # print(f"p-value: {p_value:.3f}")
        session_name_list.append(session_name)
        avg_effect_list.append(avg_effect)
        avg_effect_lower_list.append(avg_effect_lower)
        avg_effect_upper_list.append(avg_effect_upper)
        p_value_list.append(p_value)
        mean_lower_upper_list.append(mean_lower_upper)
    
    # print(len(session_name_list))
    if df is None:
        data_dict = {f"Playback Session": session_name_list}
        df = pd.DataFrame(data_dict)
    
    data_dict = {
        f"{data_name}": mean_lower_upper_list,
        f"{data_name}_avg_effect": avg_effect_list,
        f"{data_name}_avg_effect_lower": avg_effect_lower_list,
        f"{data_name}_avg_effect_upper": avg_effect_upper_list,
        f"{data_name}_p_value": p_value_list,
    }
    df_tmp = pd.DataFrame(data_dict)
    # print(len(df))
    df = pd.concat([df, df_tmp], axis=1)

In [ ]:
display(df[["Playback Session", "S-VeDBA", "AD-Speed", "AD-MPR"]])